In [2]:
import cv2
import numpy as np
import tensorflow as tf

model_path = "model.keras"
model = tf.keras.models.load_model(model_path)
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)   
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)   

cv2.namedWindow("Digit Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Digit Detection", 1280, 720)

print("Show digit photos to the camera. Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    h, w, _ = frame.shape
    size_w = int(w * 0.6)
    size_h = int(h * 0.6)
    x1 = w // 2 - size_w // 2
    y1 = h // 2 - size_h // 2
    x2 = x1 + size_w
    y2 = y1 + size_h

    roi = frame[y1:y2, x1:x2]

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28))
    normalized = resized.astype("float32") / 255.0
    input_data = np.expand_dims(normalized, axis=(0, -1))

    preds = model.predict(input_data, verbose=0)
    digit = np.argmax(preds)
    confidence = np.max(preds)

    if confidence > 0.7:
        result_text = f"DETECTED: {digit}"
        conf_text = f"Confidence: {confidence:.2f}"
        color = (0, 255, 0)
        cv2.putText(frame, result_text, (50, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
        cv2.putText(frame, conf_text, (50, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    else:
        cv2.putText(frame, "Searching for digit...", (50, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
    cv2.putText(frame, "Inside blue rectangle", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Digit Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
print("Detection stopped.")
 

Show digit photos to the camera. Press 'q' to quit
Detection stopped.
